<a href="https://colab.research.google.com/github/cataluna84/VisionInterpretability/blob/main/notebooks/cataluna84__segment_3_dataset_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🚀 Environment Setup (Run this cell first!)
# @markdown This cell sets up the environment for both Colab and local runs.

import sys
import os

# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Running in Google Colab")
    
    # Clone the repository if not already cloned
    if not os.path.exists('VisionInterpretability'):
        !git clone https://github.com/cataluna84/VisionInterpretability.git
    
    # Change to project directory
    os.chdir('VisionInterpretability')
    
    # Install dependencies
    !pip install -q torch torchvision matplotlib numpy pillow tqdm opencv-python requests torch-lucent
    
    # Add src to Python path
    sys.path.insert(0, 'src')
    print("✅ Colab setup complete!")
else:
    print("💻 Running locally")
    # For local runs, add src to path if running from notebooks directory
    if os.path.basename(os.getcwd()) == 'notebooks':
        sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))
    elif 'src' not in sys.path:
        sys.path.insert(0, 'src')
    print("✅ Local setup complete!")

# Detailed Analysis: Feature Visualization & Dataset Examples
## InceptionV1 - Layer mixed4a

This notebook provides a rigorous exploration of **INTERPRETABILITY** in Convolutional Neural Networks (CNNs). we focus on the `mixed4a` layer of InceptionV1 (GoogLeNet).

### The Goal
We aim to answer: **"What features is the network looking for?"**
We use two complementary approaches:
1.  **Dataset Examples**: Finding real images from the dataset (ImageNet) that maximally activate a specific neuron.
    *   *Question:* "What real-world patterns trigger this neuron?"
2.  **Feature Visualization (Optimization)**: Synthesizing an "ideal" input image that maximizes the neuron's activation using gradient ascent.
    *   *Question:* "What is the 'platonic ideal' of the feature this neuron detects?"

---
### Theoretical Foundations

#### 1. Forward Pass & Activation
A neural network is a function $f: X \to Y$.
For an input image $x$, the activation $A_{n,l}(x)$ represents the output of the $n$-th neuron at layer $l$.
We refer to this map $A_{n,l}(x)$ as the "Feature Map".

#### 2. Optimization (Feature Visualization)
To visualize what a neuron wants, we perform **Optimization on the Input**.
Unlike training (where we update weights $w$ to minimize loss), here we freeze the weights and update the image $x$.

**The Objective:**
$$ x^* = \arg\max_x (A_n(x) - \lambda R(x)) $$

Where:
*   $A_n(x)$: The activation of the target neuron.
*   $R(x)$: Regularization terms (e.g., total variation, frequency penalties) to ensure the image looks "natural" and not just high-frequency adversarial noise.

**The Update Rule (Gradient Ascent):**
$$ x_{t+1} = x_t + \eta \nabla_x (A_n(x_t) - \lambda R(x_t)) $$

We use the **Lucent** library (PyTorch port of Lucid) which implements robust optimization techniques:
*   **Fourier Parameterization**: Optimizing in frequency space (decorrelated) speeds up convergence.
*   **Transformation Robustness**: We maximize the expected activation under random jitters/scales: $E_{t \sim T}[A_n(t(x))]$. This forces the feature to be robust and identifiable.

---


## 1. Setup & Configuration

We initialize the environment, load the pre-trained InceptionV1 model, and configure our experiment parameters.
We use `wandb` to log our experiments for reproducibility.


In [ ]:
import os
import sys
from pathlib import Path

# Add src to path for local imports
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Configuration
CONFIG = {
    "num_neurons": 10,           # First 10 neurons of mixed4a (512 total)
    "samples_per_category": 9,    # 9 samples per category (min, slight-, slight+, max)
    "max_samples": None,         # Process ALL samples (WARNING: ~1.28M images)
                                 # Set to 1000 or 10000 for faster testing.
    "batch_size": 1024,             # Batch size for streaming inference
    "layer_name": "mixed4a",      # Target layer in InceptionV1
    "wandb_project": "vision-interpretability",
    "wandb_run_name": "mixed4a-activation-spectrum-v3",
    "generate_optimized": True,   # Generate gradient-ascent visualizations
}

print("Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")


In [ ]:
import wandb

# Initialize WANDB run
run = wandb.init(
    project=CONFIG["wandb_project"],
    name=CONFIG["wandb_run_name"],
    config=CONFIG,
)

print(f"WANDB Run: {run.url}")

## 2. Theory: Activation Extraction

We load the model and attach hooks.

In [ ]:
from lucent.modelzoo import inceptionv1

# Load pretrained InceptionV1
model = inceptionv1(pretrained=True)
model.to(device).eval()

print("InceptionV1 loaded successfully!")
print(f"\nModel layers containing 'mixed4a':")
for name, module in model.named_modules():
    if "mixed4a" in name:
        print(f"  {name}: {type(module).__name__}")

In [ ]:
from segment_3_dataset_images.activation_pipeline import (
    ActivationExtractor,
    ActivationSpectrumTrackerV2,
    SampleRecord,
)

# Register forward hook on mixed4a
extractor = ActivationExtractor(model, CONFIG["layer_name"])

# Initialize spectrum tracker
tracker = ActivationSpectrumTrackerV2(
    num_neurons=CONFIG["num_neurons"],
    samples_per_category=CONFIG["samples_per_category"],
)

print(f"Tracking {CONFIG['num_neurons']} neurons with {CONFIG['samples_per_category']} samples per category")

## 2. Theory: Dataset Processing (Max-Min Search)

To find **Dataset Examples**, we scan the entire ImageNet dataset (1.2M images).
For each image $x_i$, we compute the activation map $A(x_i)$.
Since $A(x_i)$ is spatial ($H \times W$), we take the maximum value across spatial dimensions:
$$ a_i = \max_{h,w} A(x_i)_{h,w} $$

We track four categories of samples for each neuron:
1.  **Maximum ($a_i \gg 0$)**: The images that excite the neuron the most.
2.  **Slightly Positive ($a_i > 0$)**: Weak activation.
3.  **Slightly Negative ($a_i < 0$)**: Weak inhibition.
4.  **Minimum ($a_i \ll 0$)**: The images that maximally suppress the neuron (if the activation function allows negative values, e.g., before ReLU, or if considering the pre-activation).

**Streaming vs. Downloading**:
The dataset is huge (~150GB). We use **HuggingFace Streaming** to process images on-the-fly without downloading the full dataset to disk. Ideally, this allows inspection of massive datasets with limited storage.


In [ ]:
from segment_3_dataset_images.activation_pipeline import ImageNetStreamer

# Setup streaming from HuggingFace
streamer = ImageNetStreamer(
    batch_size=CONFIG["batch_size"],
    max_samples=CONFIG["max_samples"],
)

max_samples_str = f"{CONFIG['max_samples']:,}" if CONFIG['max_samples'] is not None else "ALL"
print(f"Streaming up to {max_samples_str} samples from ImageNet-1k")
print(f"Batch size: {CONFIG['batch_size']}")

if CONFIG['max_samples'] is not None:
    print(f"Estimated batches: {CONFIG['max_samples'] // CONFIG['batch_size']}")
else:
    print("Estimated batches: Unknown (streaming infinite/all)")

## 3. Batch Processing Loop

This is the core execution loop.
*   **Input**: Stream of ImageNet batches.
*   **Process**:
    1.  **Inference**: Forward pass through InceptionV1.
    2.  **Extraction**: `ActivationExtractor` hooks grab the tensor at `mixed4a`.
    3.  **Tracking**: `ActivationSpectrumTracker` updates the "Top-K" and "Bottom-K" lists using a min-heap algorithm for efficiency.
*   **Logging**:
    We log **throughput** (images/sec) to WandB to monitor performance. Since streaming can be network-bound, this metric helps diagnose bottlenecks.


In [ ]:
# Process batches with TIMING and LOGGING
import time

total_processed = 0
batch_count = 0

print(f"Processing batches using device: {device}...")
t_start_total = time.time()

for tensors, images, labels, ids in tqdm(streamer, desc="Batches"):
    t_batch_start = time.time()
    
    # 1. Move Data to GPU (Bottleneck: Host-to-Device transfer)
    tensors = tensors.to(device)
    
    # 2. Forward Pass (Inference)
    # hooks in ActivationExtractor capture the data automatically
    with torch.no_grad():
        _ = model(tensors)
    
    # 3. Extract Activations
    # get_max_activations_per_channel performs Global Max Pooling (B, C, H, W) -> (B, C)
    activations = extractor.get_max_activations_per_channel()
    
    # 4. Update Tracker (CPU)
    # We move activations back to CPU for sorting/tracking to avoid GPU sync stalls
    tracker.update(
        activations[:, :CONFIG["num_neurons"]],
        images,
        ids,
        labels,
    )
    
    # Timing calculations
    batch_duration = time.time() - t_batch_start
    current_batch_size = len(tensors)
    throughput = current_batch_size / batch_duration
    
    total_processed += current_batch_size
    batch_count += 1
    
    # 5. Logging to WandB
    # We allow the user to monitor performance in real-time
    if batch_count % 10 == 0:
        wandb.log({
            "batch": batch_count,
            "samples_processed": total_processed,
            "batch_duration_sec": batch_duration,
            "throughput_img_per_sec": throughput,
        })

t_end_total = time.time()
print(f"\nProcessed {total_processed:,} samples in {batch_count} batches")
print(f"Total time: {t_end_total - t_start_total:.2f} seconds")
print(f"Average throughput: {total_processed / (t_end_total - t_start_total):.2f} img/sec")


## 4. Generating "Dream" Images (Optimization)

Here we generate the **Synthetic Examples**.
We use `torch-lucent` to perform the gradient ascent described in the Theory section.

**Why Generate?**
Dataset examples can only show us *correlation*. If a neuron activates for a "dog head", is it detecting the *eyes*, the *snout*, or the *fur texture*?
Optimization isolates the causal feature. If the optimized image contains *only* a snout pattern, we know the neuron is a "snout detector".

**Negative Optimization**:
We also optimize for the *negative* objective:
$$ x^*_{neg} = \arg\min_x A_n(x) = \arg\max_x (-A_n(x)) $$
This shows us what pattern the neuron *actively avoids* or is inhibited by.


In [ ]:
from lucent.optvis import render, objectives

optimized_images = {}  # Positive optimization (maximize activation)
optimized_negative_images = {}  # Negative optimization (minimize activation)

if CONFIG["generate_optimized"]:
    print(f"Generating optimized examples for {CONFIG['num_neurons']} neurons...")
    print("This generates TWO images per neuron:")
    print("  - Positive: What the neuron WANTS to see (maximize activation)")
    print("  - Negative: What the neuron AVOIDS (minimize activation)")
    print()
    
    for n in tqdm(range(CONFIG["num_neurons"]), desc="Optimizing"):
        # --- Positive Optimization (maximize activation) ---
        try:
            objective = f"{CONFIG['layer_name']}:{n}"
            result = render.render_vis(
                model,
                objective,
                show_image=False,
                show_inline=False,
                thresholds=(512,),
            )
            
            if result and len(result) > 0:
                img_array = result[0][0]
                img_array = (img_array * 255).astype(np.uint8)
                optimized_images[n] = Image.fromarray(img_array)
            else:
                optimized_images[n] = None
                
        except Exception as e:
            print(f"  Failed positive optimization for neuron {n}: {e}")
            optimized_images[n] = None
        
        # --- Negative Optimization (minimize activation) ---
        try:
            # Create a negated channel objective using lucent's objectives API
            positive_objective = objectives.channel(CONFIG['layer_name'], n)
            negative_objective = -1 * positive_objective
            
            result = render.render_vis(
                model,
                negative_objective,
                show_image=False,
                show_inline=False,
                thresholds=(512,),
            )
            
            if result and len(result) > 0:
                img_array = result[0][0]
                img_array = (img_array * 255).astype(np.uint8)
                optimized_negative_images[n] = Image.fromarray(img_array)
            else:
                optimized_negative_images[n] = None
                
        except Exception as e:
            print(f"  Failed negative optimization for neuron {n}: {e}")
            optimized_negative_images[n] = None
    
    pos_count = sum(1 for v in optimized_images.values() if v is not None)
    neg_count = sum(1 for v in optimized_negative_images.values() if v is not None)
    print(f"Generated {pos_count} positive and {neg_count} negative optimized images")
else:
    print("Skipping optimized example generation")

## 5. Visualization: The Distill Spectrum

We visualize the results using the layout pioneered by Distill.pub.
The **Activation Spectrum** visualization arranges samples from "Most Negative" to "Most Positive".

**Layout:**
| Neg Optimized | Min Dataset | Slight Neg | Slight Pos | Max Dataset | Pos Optimized |
| :---: | :---: | :---: | :---: | :---: | :---: |
| *What it hates* | *Real suppressant* | *Weak suppressant* | *Weak trigger* | *Real trigger* | *What it loves* |

This provides a holistic view of the neuron's behavior, verifying if the synthetic "dream" matches the real-world triggers.


In [ ]:
from segment_3_dataset_images.visualization import plot_neuron_spectrum_distill

# The plot_neuron_spectrum_distill function creates Distill.pub style visualizations
# showing the full activation spectrum:
# | Neg Optimized | Min Grid | Slight- | Slight+ | Max Grid | Pos Optimized |
print(f"Using Distill.pub style visualization for {CONFIG['num_neurons']} neurons")

In [ ]:
# Visualize spectrum for all tracked neurons using Distill.pub style
print(f"Visualizing activation spectrum for {CONFIG['num_neurons']} neurons...\n")

figures = []
for n in range(CONFIG["num_neurons"]):
    spectrum = tracker.get_spectrum(n)
    pos_img = optimized_images.get(n)
    neg_img = optimized_negative_images.get(n)
    
    # Use Distill.pub style visualization from the visualization module
    fig = plot_neuron_spectrum_distill(
        neuron_idx=n,
        layer_name=CONFIG["layer_name"],
        spectrum=spectrum,
        optimized_img=pos_img,
        negative_optimized_img=neg_img,
    )
    figures.append(fig)
    plt.show()
    print()

## 6. Saving Results

We save the generated visualizations to the local file system for further analysis.

In [ ]:
import os

# Create results directory
results_dir = project_root / "notebooks" / "results" / "segment_3_dataset_images"
results_dir.mkdir(parents=True, exist_ok=True)
print(f"Saving results to: {results_dir}")

# Initialize WandB Table for logging chart
wandb_table = wandb.Table(columns=["Neuron Index", "Activation Spectrum"])

if 'figures' in locals() and figures:
    for n, fig in enumerate(figures):
        # Save locally
        save_path = results_dir / f"neuron_{n}_spectrum.png"
        fig.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"  Saved: {save_path.name}")
        
        # Add to WandB Table
        # converting figure to image for logging
        try:
            wandb_table.add_data(n, wandb.Image(fig))
        except Exception as e:
            print(f"  Warning: Could not add figure {n} to WandB table: {e}")
else:
    print("No figures found to save.")

# Log the table to WandB
if wandb.run is not None:
    wandb.log({"activation_spectrum_chart": wandb_table})
    print("Logged activation spectrum table to WandB.")

## 7. Conclusion

In this notebook, we explored the `mixed4a` layer of InceptionV1 using two complementary techniques:
1.  **Dataset Examples**: We found real images that maximally activate specific neurons. This tells us *what* the neuron responds to in the real world.
2.  **Feature Visualization**: We synthesized 'dream' images via gradient ascent. This tells us the *purest form* of the feature the neuron detects.

**Key Takeaways:**
*   By comparing the 'Dataset Examples' with the 'Optimized' images, we can verify if our interpretation of the neuron is correct.
*   If the dataset examples match the optimized feature (e.g., both look like 'dog heads'), we discovered a robust feature.
*   If they differ, the neuron might be polysemantic (detecting multiple unrelated concepts) or responding to context/background correlations.

This 'Spectrum' visualization (Dataset Min/Max + Optimization) is a powerful tool for rigorous interpretability.

## 8. Cleanup

We finish the WandB run to ensure all logs are synced.

In [ ]:
# Finish WandB run
if wandb.run is not None:
    wandb.finish()
    print("WandB run finished.")
